# Extracting Data from LMC and Vicon

In [3]:
%load_ext autoreload
%autoreload 2
# the imported files are updated each time a cell is executed (good if functions in py files are added and modified)
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os

from LMC_Vicon_util import create_transformation_matrix, alignment
from LMC_Vicon_util import fuse_signals, calculate_length_angle, calculate_visibility
from LMC_Vicon_util import save_length_angle, save_length_angle_visib
from LMC_Vicon_util import readFile_trc, convert_vicon_to_lmc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# General Settings
# Data Path
current_dir = os.getcwd()
data_path = 'data/_process_'
save_path = 'data/_result_'

# number of files to process
num_files = 1

# configuration of Naiv LMCs, rotation in degree
naiv_config = [[-60, 0, 60, 0, 0, 0],
               [60, 0, 60, 0, 0, 0],
               [-60, 0, -60, 0, 0, 0],
               [60, 0, -60, 0, 0, 0]]
# with offset against sensor 1
naiv_config_offset = [[0, 0, 0, 0, 0, 0],
                     [-12.92, 18.79, -21.65, 0, 0, 0],
                     [-7.75, 17.65, -4.81, 0, 0, 0],
                     [6.0, -5.92, -6.42, 0, 0, 0]]

# configuration of Opti LMCs, rotation in degree
opti_config = [[-120.37, 0, -256.29, 29.90, 16.50, 0],
               [-69.97, 0, 342.57, -7.57, -7.04, 0],
               [-190.60, 0, 88.70, -8.72, -4.93, 0],
               [178.88, 0, 100.90, -12.06, 12.38, 0]]
# with offset against sensor 3
opti_config_offset = [[0.54, 5.33, 18.36, 0, 0, 0],
                     [-22.91, -23.19, -82.96, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0],
                     [-11.28, -39.85, 8.85, 0, 0, 0]]

# calculation of naiv and optimized transformation matrix
naiv_trans_matrix = np.zeros((4,4,4))
opti_trans_matrix = np.zeros((4,4,4))
for i in range (0,4):
  naiv_trans_matrix[i] = create_transformation_matrix(naiv_config[i])
  opti_trans_matrix[i] = create_transformation_matrix(opti_config[i])

naiv_offset_matrix = np.zeros((4,4,4))
opti_offset_matrix = np.zeros((4,4,4))
for i in range (0,4):
  naiv_offset_matrix[i] = create_transformation_matrix(naiv_config_offset[i])
  opti_offset_matrix[i] = create_transformation_matrix(opti_config_offset[i])

# PROCES DATA
for subject in os.listdir(os.path.join(os.getcwd(),  'data', '_process_')):
  if os.path.isdir(os.path.join(os.getcwd(),  'data', '_process_', subject)):
    print('_________________________________________________________________________')
    print(subject)
    for trialtype in ['Naiv', 'Opti']:
      print('    ', trialtype)
      data_path = os.path.join(os.getcwd(),  'data', '_process_', subject, trialtype)
      save_path = os.path.join(os.getcwd(),  'data', '_result_')
      save_filename = os.path.join(save_path,subject+"_"+trialtype)

      # create file to hold the statistic per trialtype
      with open(save_filename+".txt", "w"):
        pass

      # list existing file
      dataList = np.array(os.listdir(os.path.join(current_dir, data_path)))      
    
      # load files
      for i in range(0,num_files):
        lmc_filename = dataList[i]
        vicon_filename = dataList[num_files+i]
        
        #load lmc file from .pkl
        lmc_filepath = os.path.join(current_dir, data_path, lmc_filename)
        df_LMC = pd.read_pickle(lmc_filepath)

        #load vicon file from .trc        
        vicon_filepath = os.path.join(current_dir, data_path, vicon_filename)
        markerList, marker_xyz, timeseries, frameseries, DataRate, data_dict = readFile_trc(vicon_filepath)
        vicon_data = convert_vicon_to_lmc (marker_xyz)

        # Set the shortest data as maximum row limit
        limitdata = int(df_LMC.shape[0]/4)
        if limitdata > vicon_data.shape[0] :
          limitdata = vicon_data.shape[0]
        df_LMC = df_LMC.iloc [0:(4*limitdata),:]
        vicon_data = vicon_data[0:limitdata,:,:]
        
        print('        {} \t: {}'.format(lmc_filename, df_LMC.shape))
        print('        {} \t\t: {}'.format(vicon_filename, vicon_data.shape))

        '''LMC Processing'''
        # save column name of read pickle for future use
        column_names = df_LMC.columns

        LMC_data = df_LMC.to_numpy()

        # assign data to ndarray, select axis for calculation
        LMC_data_device = np.zeros((4,int(LMC_data.shape[0] / 4)  ,99))
        LMC_marker = np.zeros((4,int(LMC_data.shape[0] / 4)  ,90))
        LMC_visib = np.zeros((4, int(LMC_data.shape[0] / 4), 5))

        ids = [1, 2, 3, 4]
        for id_val in ids:
          LMC_data_device[id_val-1,:,:] = LMC_data[LMC_data[:, 0] == id_val] #filter to each device
          LMC_marker[id_val-1,:,:] = LMC_data_device[id_val-1,:, 4 : 94]    #select only marker
          LMC_visib[id_val-1,:,:] = LMC_data_device[id_val-1,:, 94 : 99]    #select only visibility

        #reshape marker to shape: (numLMC, numRowData, marker_name, x_y_z)
        LMC_marker = LMC_marker.reshape(LMC_marker.shape[0], LMC_marker.shape[1], int(LMC_marker.shape[2]/3), 3)

        #calculation of alignment all the 4 LMCs
        if "Naiv" in lmc_filename:
          LMC_marker = alignment(LMC_marker,naiv_trans_matrix)
          LMC_marker = alignment(LMC_marker,naiv_offset_matrix)
        elif "Opti" in lmc_filename:
          LMC_marker = alignment(LMC_marker,opti_trans_matrix)
          LMC_marker = alignment(LMC_marker,opti_offset_matrix)

        # save marker after alignment for plotting to fix variable
        marker_LMC_plot = LMC_marker

        ### Return the transformation calculation ndarray to initial file format
        LMC_marker = LMC_marker.reshape(LMC_marker.shape[0],LMC_marker.shape[1],
                                        LMC_marker.shape[2]*LMC_marker.shape[3])
        for id_val in ids:
          LMC_data_device[id_val-1,:, 4 : 94] = LMC_marker[id_val-1,:,:]
          LMC_data[LMC_data[:, 0] == id_val] =LMC_data_device[id_val-1,:,:]

        # KALMAN FILTER
        # create empty ndarray for each LMC to be converted to dataframe
        sigLMC1 = np.zeros((int(LMC_data.shape[0]/4),LMC_data.shape[1]))
        sigLMC2 = np.zeros((int(LMC_data.shape[0]/4),LMC_data.shape[1]))
        sigLMC3 = np.zeros((int(LMC_data.shape[0]/4),LMC_data.shape[1]))
        sigLMC4 = np.zeros((int(LMC_data.shape[0]/4),LMC_data.shape[1]))

        # create empty dataframe for fused signal
        df_fused_LMC = pd.DataFrame(columns = column_names)
        df_fused_LMC = df_fused_LMC.iloc[:,[3] + [i for i in range (4,94)]]
        column_Kalman = df_fused_LMC.columns

        # select data for Kalman Filter, timestamp and markers, omit visibility
        sigLMC1 = LMC_data[LMC_data[:, 0] == 1]
        sigLMC1 = sigLMC1 [:,[3] + [i for i in range (4,94)]]
        sigLMC2 = LMC_data[LMC_data[:, 0] == 2]
        sigLMC2 = sigLMC2 [:,[3] + [i for i in range (4,94)]]
        sigLMC3 = LMC_data[LMC_data[:, 0] == 3]
        sigLMC3 = sigLMC3 [:,[3] + [i for i in range (4,94)]]
        sigLMC4 = LMC_data[LMC_data[:, 0] == 4]
        sigLMC4 = sigLMC4 [:,[3] + [i for i in range (4,94)]]

        # create dataframe for kalman calculation
        df_sigLMC1 = pd.DataFrame(sigLMC1, columns= column_Kalman)
        df_sigLMC2 = pd.DataFrame(sigLMC2, columns= column_Kalman)
        df_sigLMC3 = pd.DataFrame(sigLMC3, columns= column_Kalman)
        df_sigLMC4 = pd.DataFrame(sigLMC4, columns= column_Kalman)

        # copy timestamp of fusedLMC from one of LMC
        df_fused_LMC.iloc[:,0] = df_sigLMC1.iloc[:,0]

        # Kalman Filter Calculation from column 1, column 0 is timestamp 
        for i in range(1,df_sigLMC1.shape[1]):
          x_arr = fuse_signals(df_sigLMC1.iloc[:,i],df_sigLMC2.iloc[:,i],df_sigLMC3.iloc[:,i],df_sigLMC4.iloc[:,i])
          df_fused_LMC.iloc[:,i] = x_arr[:,0]

        # Convert Kalman Result to ndarray
        fused_LMC = df_fused_LMC.to_numpy()

        # Select only marker
        fused_LMC_marker = fused_LMC [:,1:91]
        fused_LMC_marker = fused_LMC_marker.reshape(fused_LMC_marker.shape[0],int(fused_LMC_marker.shape[1]/3),3)

        # Calculate Visibility
        visibility = np.zeros((4,4,5))
        visibility = calculate_visibility(LMC_visib)

        # Calculate Finger Length, Angle Joint and Segment Length
        lmc_finger_length = np.zeros ((fused_LMC_marker.shape[0],5))
        lmc_segment_length = np.zeros ((fused_LMC_marker.shape[0],14))
        lmc_angle_joint = np.zeros ((fused_LMC_marker.shape[0],6))
        lmc_finger_length, lmc_angle_joint, lmc_segment_length = calculate_length_angle (fused_LMC_marker)

        '''VICON Processing'''
        vic_finger_length = np.zeros ((vicon_data.shape[0],5))
        vic_segment_length = np.zeros ((vicon_data.shape[0],14))
        vic_angle_joint = np.zeros ((vicon_data.shape[0],6))
        vic_finger_length, vic_angle_joint, vic_segment_length = calculate_length_angle (vicon_data)

        '''SAVING DATA'''
        save_length_angle_visib (save_filename, lmc_filename, lmc_finger_length, lmc_angle_joint, lmc_segment_length, visibility)
        save_length_angle(save_filename, vicon_filename, vic_finger_length, vic_angle_joint, vic_segment_length)
        

_________________________________________________________________________
Subject1
     Naiv
        Interp_LMC_Naiv_Static101.pkl 	: (200, 99)
        Naiv_Static101.trc 		: (50, 30, 3)
     Opti
        Interp_LMC_Opti_Static101.pkl 	: (520, 99)
        Opti_Static101.trc 		: (130, 30, 3)
